## Generación de datos y entrenamiento del modelo

Este experimento consiste en dado un set de 10,000 datos históricos de peticiones de un usuario a la aplicación ABC Jobs, vamos a entrenar un modelo de aprendizaje de máquina para que 'aprenda' -valga la redundancia- el comportamiento normal de un usuario. 

Después utilizaremos este modelo para cada vez que recibamos una petición, el componente detector de intrusos determine si el comportamiento de la petición es normal o si se trata de una anomalía y por ende debe generar una alerta que será enviada al componente notificaciones management, que simulará la notificación a un usuario a través de un log.

Finalmente realizaremos un análisis de la veracidad de la táctica implementada a través de 100 peticiones y simularemos en 3 de ellas un ataque.

Para efectos del experimento, los 10,000 datos históricos serán construídos siguiendo heuristicas que más adelante explicaremos.

### 1. Mapa de atributos

El tipo de petición con el que vamos a trabajar tiene la siguiente estructura:

```python
req: {'http_method': 'delete', 'url_endpoint': '/ofertas', 'user_agent': 'firefox', 'operating_system': 'macos', 'ip_address': '127.5.87.126', 'access_datetime': '2022-03-19T21:55:25'}
```

A continuación definimos un mapa de atributos de una petición a números enteros que serán procesados por nuestro modelo.



In [1]:
REQ_MAPPING = {
    'http_method': {
        'get': 1,
        'post': 2,
        'put': 3,
        'delete': 4,
        'other': 5
    },
    'url_endpoint': {
        '/ofertas': 1,
        '/pagos': 2,
        '/contratos': 3,
        'other': 4
    },
    'user_agent': {
        'safari': 1,
        'chrome': 2,
        'firefox': 3,
        'other': 4
    },
    'ip_address': {
        '127.5.87.255': 1, # home address
        '127.5.87.126': 2, # work address
        'other': 3
    },
    'operating_system': {
        'macos': 1,
        'windows': 2,
        'other': 3,
    },
    'time': {
        'work_hours': 1, # 8 am - 5 pm
        'after_hours': 2, # 5 pm - 10 pm
        'off_hours': 3, # 10 pm - 8 am
    },
    'day_of_week': {
        'monday': 1,
        'tuesday': 2,
        'wednesday': 3,
        'thursday': 4,
        'friday': 5,
        'saturday': 6,
        'sunday': 7
    }
}


Siguiente con el ejemplo de la petición de ejemplo mostrada anteriormente el objeto mapeado que utilizará nuestro modelo se vería así:

```python
mapping: {'http_method': 4, 'url_endpoint': 1, 'user_agent': 3, 'operating_system': 1, 'ip_address': 2, 'time': 2, 'day_of_week': 6}
```

### 2. Función que hará el mapeo.

Esta función tendrá la responsabilidad de convertir un diccionario de atributos de una petición en un diccionario mapeado a los enteros que espera nuestro modelo.

In [2]:
from datetime import datetime, time, timedelta

def map_datetime(datetime_str, mapping)->tuple:
    """
    Maps the input datetime string to corresponding time and day_of_week categories.
    
    Parameters:
    - datetime_str (str): The datetime string in ISO 8601 format to be mapped.
    - mapping (dict): The mapping dictionary containing time and day_of_week categories.
    
    Returns:
    - tuple: A tuple containing mapped time and mapped day_of_week.
    """
    # Parse the datetime string to a datetime object
    dt = datetime.fromisoformat(datetime_str)
    
    # Extract the time and day of the week
    t = dt.time()
    day_of_week = dt.strftime('%A').lower()
    
    # Map the time to the corresponding category
    if time(8, 0) <= t <= time(17, 0):
        mapped_time = mapping['time']['work_hours']
    elif time(17, 0) < t <= time(22, 0):
        mapped_time = mapping['time']['after_hours']
    else:
        mapped_time = mapping['time']['off_hours']
    
    # Map the day of the week to the corresponding category
    mapped_day_of_week = mapping['day_of_week'][day_of_week]
    
    return mapped_time, mapped_day_of_week

def get_request_mapping(req: dict)->dict:
    """
    Maps the features of the input request dictionary to corresponding categories using a predefined mapping.
    
    Parameters:
    - req (dict): The input request dictionary containing features to be mapped.
    
    Returns:
    - dict: A dictionary containing the mapped features.
    """
    mapped_dict = {}
    # Loop over the req dict
    for feature, value in req.items():
        if feature != 'access_datetime':
            mapped_feature = REQ_MAPPING[feature].get(value) if REQ_MAPPING[feature].get(value) is not None else REQ_MAPPING[feature]['other']
            mapped_dict[feature] = mapped_feature
        else:
            mapped_time, mapped_day_of_week = map_datetime(value, REQ_MAPPING)
            mapped_dict['time'] = mapped_time
            mapped_dict['day_of_week'] = mapped_day_of_week
    return mapped_dict
            
        
        

### 3. Generación de datos aleatorios de un usuario

A continuación vamos a generar 10,000 datos aleatorios de un usuario administrador de una empresa. Las heurísticas empleadas para la generación se explican a continuación.

### http method

El usuario admin acostumbra a realizar consultas (**get**), a hacer publicaciones de ofertas (**post**), no suele realizar muchos cambios a ofertas ya publicadas (**put**) y rara vez borra ofertas (**delete**).

|http_method|prob|
|-----------|----|
|get|50%|
|post|35%|
|put| 10%|
|delete|5%|

### url endpoint

El usuario admin acostumbra a consumir 3 rutas la mayor parte de su tiempo, **ofertas**, **pagos** y **contratos**; pero, también puede consumir otras rutas de la aplicación que las denotaremos como **dummy_route**.

|url_endpoint|prob|
|-----------|----|
|/ofertas|30%|
|/pagos|30%|
|/contratos|10%|
|/dummy_route|10%|

### time

El usuario tiene horario de oficina normal de **8:00 am a 5:00 pm** y el 95% de las peticiones realizadas se encuentran en este rango. Así mismo algunas veces (4%) suele trabajar **hasta las 10:00 pm** y escasamente trabaja **después de las 10:00 pm**, cuando le toca presentar algún informe para el día siguiente y no se encuentra preparado (1%).

|time|prob|
|-----------|----|
|08:00 am - 05:00 pm|95%|
|05:00 pm - 10:00 pm|4%|
|10:00 pm - 08:00 am|1%|

### user_agent

El usuario es amante de Apple y todos sus productos y servicios, por tal motivo utiliza **safari** (95%) como su navegador principal tanto en casa como en oficina; sin embargo, esporádicamente tiene que entrar desde otros navegadores como **chrome** (3%) o **firefox** (1%), e incluso ha utilizado otros navegadores como edge u opera los cuales denotaremos como **dummy_agent** (1%), cuando no tiene acceso a su máquina habitual.

|user_agent|prob|
|-----------|----|
|safari|95%|
|chrome|3%|
|firefox|1%|
|dummy_agent|1%|

### ip_address

Dentro de nuestra base de datos tenemos registrada la dirección ip de la casa y de la de oficina del usuario. Así mismo sabemos que el usuario ingresa a la aplicación desde su **casa** el 20% de las veces, desde su **oficina** el 78% de las veces y desde **otras direcciones** el 2% de las veces (simulando situaciones en que el usuario está de vacaciones y necesita acceder a la aplicación).

|user_agent|prob|
|-----------|----|
|casa: 127.5.87.255 |20%|
|oficina: 127.5.87.126 |78%|
|otros|2%|

### operating_system

Como ya vimos, el usuario es amante de Apple por ende utiliza un macbook pro (**macos**) para ingresar a la aplicación el 95% de las veces; sin embargo, a veces no tiene su máquina a la mano y utiliza **windows** para su navegación en la aplicación. Esto ocurre el 5% de las veces. El usuario no se siente cómodo con distribuciones de Linux a pesar de que utiliza **macos**, así que no utiliza ningún sistema operativo adicional.

|operating_system|prob|
|-----------|----|
|macos|95%|
|windows|5%|

### day_of_week

Nuestro usuario al tener horario de oficina, trabaja de **lunes a viernes** el 95% de las veces (la distribución entre los 5 días de la semana es uniforme); no obstante, algunas veces tiene que trabajar los **sábados** con probabilidad del 4% y muy rara vez le ha tocado trabajar un **domingo** para ponerse al día en sus tareas (1%).

|day_of_week|prob|
|-----------|----|
|lunes a viernes|95%|
|sábado|4%|
|domingo|1%|

A continuación presentamos las funciones encargadas de la generación de un dato y de los 10,000 datos siguiendo las heurísticas aquí presentadas:

#### 3.1 Generación de una petición aleatoria

In [38]:
import random

def generate_random_datetime(start_year: int, end_year: int)->str:
    """
    Generates a random datetime string within the specified year range with weighted probabilities 
    for days of the week and time ranges.
    
    Parameters:
    - start_year (int): The start year for generating random datetime.
    - end_year (int): The end year for generating random datetime.
    
    Returns:
    - str: A random datetime string in ISO 8601 format.
    """
    # Define the days of the week with corresponding weights
    days_of_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    day_weights = [19, 19, 19, 19, 19, 4, 1]  # Weights for each day of the week
    
    # Define the time ranges with corresponding weights
    time_ranges = [
        ('working_hours', 8, 17, 95),  # 8 am - 5 pm with 95% probability
        ('after_hours', 17, 22, 4),   # 5 pm - 10 pm with 4% probability
        ('off_hours', 22, 24, 0.5),   # 10 pm - midnight with 0.5% probability
        ('off_hours', 0, 8, 0.5),     # midnight - 8 am with 0.5% probability
    ]
    
    # Choose a random day of the week based on the given weights
    chosen_day = random.choices(days_of_week, weights=day_weights)[0]
    
    # Choose a random time range based on the given probabilities
    time_range = random.choices(time_ranges, weights=[weight for _, _, _, weight in time_ranges])[0]
    
    # Extract the details of the chosen time range
    _, start_hour, end_hour, _ = time_range
    
    # Generate a random hour, minute, and second within the chosen time range
    random_hour = random.randint(start_hour, end_hour - 1)
    random_minute = random.randint(0, 59)
    random_second = random.randint(0, 59)
    
    # Generate a random year, month, and day
    random_year = random.randint(start_year, end_year)
    random_month = random.randint(1, 12)
    random_day = random.randint(1, 28) # To avoid having february the 30th and the likes
    
    # Create a base date with the generated year, month, and day
    base_date = datetime(random_year, random_month, random_day)
    
    # Calculate the number of days to shift from the base date to get the chosen day of the week
    days_shift = (days_of_week.index(chosen_day) - base_date.weekday() + 7) % 7
    
    # Generate the random date and time
    random_datetime = datetime.combine(base_date.date() + timedelta(days=days_shift), time(random_hour, random_minute, random_second))
    
    # Format the datetime object to a string in ISO 8601 format
    return random_datetime.isoformat()

def generate_random_ip()->str:
    """
    Generates a random IP address string with weighted probabilities for predefined IP addresses.
    
    Returns:
    - str: A random IP address string.
    """
    # Define the IP addresses and their corresponding probabilities
    ips = ['127.5.87.255', '127.5.87.126', 'other']  # home, office, other
    weights = [20, 78, 2]
    
    # Choose an IP address according to the given probabilities
    chosen_ip = random.choices(ips, weights)[0]

    # If 'other' is chosen, generate a random IP address
    if chosen_ip == 'other':
        chosen_ip = f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}"

    return chosen_ip

def generate_random_request()->dict:
    """
    Generates a random request dictionary with weighted probabilities for each feature.
    
    The features include http_method, url_endpoint, user_agent, operating_system, 
    ip_address, and access_datetime.
    
    Returns:
    - dict: A dictionary containing randomly generated request features.
    """

    req = {}
    # Define generators for 4 features
    http_method_generator = [['get', 'post', 'put', 'delete'], [50, 35, 10, 5]]
    url_generator = [['/ofertas', '/pagos', '/contratos', '/dummy_route'],[30, 30, 30, 10]]
    user_agent_generator = [['safari', 'chrome', 'firefox', 'dummy_agent'], [95, 3, 1, 1]]
    operating_system_generator = [['macos', 'windows'], [95, 5]]

    # Randomly select a value for the 4 features
    req['http_method'] = random.choices(http_method_generator[0], weights=http_method_generator[1])[0]
    req['url_endpoint'] = random.choices(url_generator[0], weights=url_generator[1])[0]
    req['user_agent'] = random.choices(user_agent_generator[0], weights=user_agent_generator[1])[0]
    req['operating_system'] = random.choices(operating_system_generator[0], weights=operating_system_generator[1])[0]

    # Ip generator
    req['ip_address'] = generate_random_ip()

    # Datetime generator
    req['access_datetime'] = generate_random_datetime(2020, 2022)

    return req



A continuación podemos apreciar un ejemplo donde generamos una petición aletoria y su respectivo mapeo:

In [39]:
req = generate_random_request()
print('req:', req)
print('mapping:', get_request_mapping(req))

req: {'http_method': 'post', 'url_endpoint': '/ofertas', 'user_agent': 'safari', 'operating_system': 'macos', 'ip_address': '127.5.87.126', 'access_datetime': '2021-04-28T15:24:33'}
mapping: {'http_method': 2, 'url_endpoint': 1, 'user_agent': 1, 'operating_system': 1, 'ip_address': 2, 'time': 1, 'day_of_week': 3}


#### 3.2 Generación de 10,000 peticiones aleatorias

Primero generamos datos para 10,000 peticiones y los guardamos en un dataframe utilizando la librería pandas.

In [40]:
import pandas as pd

# Generate 10000 random data points
data_points = [generate_random_request() for _ in range(10000)]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_points)
df.head()


,http_method,url_endpoint,user_agent,operating_system,ip_address,access_datetime
0,get,/ofertas,safari,macos,127.5.87.126,2020-05-27T13:52:41
1,get,/pagos,safari,macos,127.5.87.126,2020-09-25T16:13:15
2,get,/contratos,safari,macos,127.5.87.126,2021-02-26T16:39:12
3,post,/contratos,safari,macos,127.5.87.126,2020-04-13T14:53:48
4,get,/dummy_route,safari,macos,127.5.87.255,2021-05-11T13:15:01


Ahora vamos a transformar el dataframe creado en el paso anterior utilizando nuestra función encargada de mapear atributos a enteros, los cuales serán después utilizados para entrenar nuestro modelo.

In [41]:
# Apply the mapping function to each row of the DataFrame
mapped_data_points = df.apply(lambda row: get_request_mapping(row.to_dict()), axis=1)

# Convert the Series of dictionaries to a DataFrame
mapped_df = pd.DataFrame(list(mapped_data_points))
mapped_df.head()

,http_method,url_endpoint,user_agent,operating_system,ip_address,time,day_of_week
0,1,1,1,1,2,1,3
1,1,2,1,1,2,1,5
2,1,3,1,1,2,1,5
3,2,3,1,1,2,1,1
4,1,4,1,1,1,1,2


### 4. Entrenamiento del modelo

Hasta el momento, hemos generado 10,000 datos que representan el histórico de peticiones realizadas por un usuario a la aplicación ABC Jobs. El siguiente paso es implementar un algoritmo de aprendizaje de máquina capaz de aprender y entender el comportamiento habitual de este usuario, para posteriormente identificar posibles anomalías que podrían indicar intentos de ataques de tipo suplantación.

Para lograr esto, nos apoyaremos en la librería scikit-learn, y haremos uso del algoritmo **Isolation Forest**. Este algoritmo es particularmente eficaz para la detección de anomalías.

Un aspecto crucial al utilizar el Isolation Forest es la definición del parámetro de contaminación. Este parámetro representa la proporción de outliers o anomalías que esperamos encontrar en el conjunto de datos. La elección de este valor es subjetiva y depende en gran medida del conocimiento del dominio y de la naturaleza de los datos. En nuestro caso, dado que hemos construido los datos basándonos en heurísticas, podemos ajustar este parámetro de acuerdo con nuestras expectativas sobre la frecuencia de anomalías.

Es importante destacar que, aunque el ajuste del parámetro de contaminación se basa en cierta medida en el juicio subjetivo, una elección informada y justificada puede mejorar significativamente la capacidad del modelo para identificar comportamientos anómalos de manera precisa.

En resumen, procederemos a entrenar nuestro modelo Isolation Forest utilizando los 10,000 puntos de datos generados, y utilizaremos un parámetro de contaminación del 1%:

In [52]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=0.01) # 1% of the data is assumed to be anomalous
model.fit(mapped_df)

IsolationForest(contamination=0.01)

A continuación vamos a generar 97 datos de peticiones del usuario siguiendo las mismas heurísticas que utilizamos para generar los 10,000 datos históricos. Así mismo vamos a inyectar 3 datos construídos manualmente simulando ataques de suplantación. Los 3 ataques tendrán las siguientes caractéristicas:

1. Los 3 ataques provendrán desde direcciones ip desconocidas.
2. Los 3 ataques buscarán cambiar, para beneficio del adversario, los valores de una oferta: método put, endpoint /ofertas.
3. En dos de los ataques, el adversario utilizará chrome como navegador y en otro utiliará safari.
4. En dos de los ataques, el adversario hará una petición desde macos y en una de ellas desde windows.
5. Un ataque se realizará un martes en horario de oficina, otro se realizará un viernes a medianoche y el último se realizará un lunes en la noche.

In [73]:
# Create 97 'actual requests' following our heuristics
user_actual_requests = [generate_random_request() for _ in range(97)]
actual_requests_df = pd.DataFrame(user_actual_requests)

# Simulate 3 attacks
simulated_attacks = [
    {
        'http_method': 'put', # Change the value of an offer
        'url_endpoint': '/ofertas', # Change the value of an offer
        'user_agent': 'chrome', # not user's usual browser
        'operating_system': 'windows', # not user's usual os
        'ip_address': f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}", # unknown ip
        'access_datetime': '2023-09-26T14:16:46' # tuesday afternoon
    },
    {
        'http_method': 'put', # Change the value of an offer
        'url_endpoint': '/ofertas', # Change the value of an offer
        'user_agent': 'safari', # user's usual browser
        'operating_system': 'macos', # user's usual os
        'ip_address': f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}", # unknown ip
        'access_datetime': '2023-09-22T23:58:46' # friday midnight
    },
    {
        'http_method': 'put', # Change the value of an offer
        'url_endpoint': '/ofertas',
        'user_agent': 'chrome', # not user's usual browser
        'operating_system': 'macos', # user's usual os
        'ip_address': f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}", # unknown ip
        'access_datetime': '2023-09-25T20:20:46' # monday night
    },
]
simulated_attacks_df = pd.DataFrame(simulated_attacks)

# Apply the mapping function to each row of the DataFrame
actual_requests_df = pd.DataFrame(list(actual_requests_df.apply(lambda row: get_request_mapping(row.to_dict()), axis=1)))
simulated_attacks_df = pd.DataFrame(list(simulated_attacks_df.apply(lambda row: get_request_mapping(row.to_dict()), axis=1)))

# Assign a label of 1 for actual user requests and -1 for attacks
actual_requests_df['label'] = 1
simulated_attacks_df['label'] = -1

# Combine dataframes
combined_data = pd.concat([actual_requests_df, simulated_attacks_df], ignore_index=True)

# Shuffle data
combined_data = combined_data.sample(frac=1).reset_index(drop=True)
combined_data.head()


,http_method,url_endpoint,user_agent,operating_system,ip_address,time,day_of_week,label
0,1,2,1,1,2,1,3,1
1,1,2,1,1,2,1,1,1
2,1,3,1,1,1,1,3,1
3,2,2,1,1,2,1,4,1
4,3,2,1,1,2,1,3,1


Ahora utilizaremos nuestro modelo entrenado para determinar de estas 100 peticiones, cuales corresponden a peticiones del usuario y cuales a ataques.

In [81]:
# Make predictions
X = combined_data.drop(columns=['label'])
y_true = combined_data['label']
y_pred = model.predict(X)



Finalmente sacaremos algunas métricas como la _accuracy_ y la _confusion matrix_ de nuestro modelo.

In [77]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy
accuracy = accuracy_score(y_true, y_pred)

print(f'Model Accuracy: {accuracy}')

Model Accuracy: 0.97


In [78]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

[[ 3  0]
 [ 3 94]]


Lo anterior significa que nuestro modelo fue acertado el 97% de las veces; sin embargo, nuestra _confusion matrix_ nos revela que 3 de los 3 ataques fueron catalogados como ataques con lo cual tenemos un 100% de ataques detectados correctamente; sin embargo, tenemos 3 peticiones normales de un usuario que fueron también catalogadas como ataques. Lo anterior sugiere que a pesar de que nuestra hipótesis de diseño se cumple, podríamos hacerlo mejor si pudiesemos entrenar nuestro modelo no solo con datos normales de comportamiento sino también con datos anómalos; es decir, tener dos etiquetas en nuestros datos, para que este pueda aprender mejor que caracteristicas sugieren un _ataque_ y que características sugieren comportamiento _normal_.

Finalmente vamos a exportar nuestro modelo para que el componente detector de intrusos puda usarlo para la generación de señales de alerta.

In [83]:
import joblib

joblib.dump(model, 'trained_model.joblib')

['trained_model.joblib']